In [49]:
import numpy as np
import cv2
import math
from PIL import Image, ImageStat
import sys
from os import listdir

In [50]:
true_po_list=[]
precision_list=[]
edge_list=[]
# K=4

In [51]:
def get_image(image_path):
    """
    Get a numpy 2D array of an image so that one can access RGBA[x][y] or RGBA[x,y].
    
    Also get a cropped image for further use for hsv and k-means clustering
    
    """
    origin = Image.open(image_path, 'r')
    width, height = origin.size
    area = (0, 0, width, 0.5*height)
    image = origin.crop(area) # crop top half of the image
#     result = Image.fromarray(image.astype(np.uint8))
    image.save(file+"_cropped.png")
    width, height = image.size
#     print (image.size)
    pixel_values = list(image.getdata())
    if image.mode == 'RGBA':
        channels = 4
    elif image.mode == 'L':
        channels = 1
    else:
        print("Unknown mode: %s" % image.mode)
        return None
#     pixel_values = np.array(pixel_values).reshape((width, height, channels))
    pixel_values = np.array(pixel_values).reshape((height, width, channels))
#     print (np.shape(pixel_values))

    return pixel_values,image,origin #return pixels matrix with RGBA value, and cropped half image

In [52]:
def get_bri(pixels):
    """
    Get a numpy 2D array of an image so that one can access brightness[x][y]
       
    """
    brightness=[[0 for x in range(len(pixels[0]))] for y in range(len(pixels))]
    # Matrix = [[0 for x in range(w)] for y in range(h)]
    for i in range(len(pixels)):
        for j in range(len(pixels[0])):
            R,G,B,A=pixels[i,j]
            brightness[i][j]=0.2126*R + 0.7152*G + 0.0722*B
    return brightness

In [53]:
def hsv( image ):
    """
    Get a numpy 2D array of an image so that one can access HSV[x][y]
       
    """
    img = np.array(image)
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     H,S,V=cv2.split(HSV)
    #H = HSV[:,:,0]
    #S = HSV[:,:,1].mean
    #V = HSV[:,:,2].mean
    #return H,S,V
#     H=np.mean(H)
#     S=np.mean(S)
#     V=np.mean(V)
    return HSV

In [54]:
"""
Return HSL matrix
"""

def hls(image):
    img = np.array(image)
    HLS = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    return HLS

In [55]:
"""
K-means clustering to segment the image into 12 color class 

clustered.png is used as output

"""
def K_means(file,K):
    
    nametemp = file

    file=(file+"_cropped.png")


    img = cv2.imread(file)
#     img = np.array(image)
    Z = img.reshape((-1,3))#reshape array into n rows and 3 colunms

    # convert to np.float32
    Z = np.float32(Z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
#     K = 15 #number of clusters
    ret,label,center=cv2.kmeans(Z,K,None,criteria,50,cv2.KMEANS_PP_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    label2=label.reshape((len(img),len(img[0]))) #save the label into 2D array corresponde to image shape
    res2 = res.reshape((img.shape))
    result = Image.fromarray(res2.astype(np.uint8))
    result.save(nametemp+"_clustered.png")
    return (label2)
# cv2.imshow('res2',res2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [56]:
"""
mark out sky
"""

def mark_sky(pixels,label2,image,file):
    #get Bmax in image
    Bmax=0
    for i in pixels:
    #     print (i)
        for j in i:
            if j[2]>Bmax:
                Bmax=j[2]

#     print (Bmax)
    

    counterclass_skypre={}
    counterclass_all={}
    Skypre=[[0 for x in range(len(pixels[0]))] for y in range(len(pixels))]
    for i in range(len(label2)):
        for j in range(len(label2[0])):
            if label2[i][j] not in counterclass_all:
                counterclass_all[label2[i][j]]=1
            else:            
                counterclass_all[label2[i][j]]+=1
    #         if label2[i][j]==0:
            #add condition like white cloud as sky
            if 0.75*180>=HLS[i][j][0]>=0.35*180 or HLS[i][j][1]>=0.95*255 \
                or (HLS[i][j][1]>0.7*255 and HLS[i][j][2]<=0.2*255)\
                or (HLS[i][j][2]<=0.005*255 and pixels[i][j][2]>=0.9*Bmax) \
                or (HLS[i][j][2]>=0.03*255 and pixels[i][j][2]>=0.9*Bmax) or \
                ((pixels[i][j][2]>=0.95*Bmax) and ((abs(pixels[i][j][2]-pixels[i][j][0])<0.02*255))):
                Skypre[i][j]=1
                if label2[i][j] not in counterclass_skypre:
                    counterclass_skypre[label2[i][j]]=1
                else:
                    counterclass_skypre[label2[i][j]]+=1

        skylabel_list=[]
        for i in counterclass_skypre:
            if counterclass_skypre[i]/counterclass_all[i]>0.7:
                skylabel_list.append(i)
#         print (skylabel_list)

    RED = 0
    GREEN = 1
    BLUE = 2

    data = np.asarray(image,dtype="int32")
#     print (np.shape(data))
    for i in range(len(pixels)):
        for j in range(len(pixels[0])):
            if label2[i][j] in skylabel_list:
                data[i, j, GREEN] = 0
                data[i, j, RED] = 0
                data[i, j, BLUE] = 255
            else:
                continue
    result = Image.fromarray(data.astype(np.uint8))
    result.save(file+"_sky_mark.png")
    
    
    
    
    """
    calculate true positive rate
    """
    true_sky_counter=0
    true_positive_counter=0
    false_positive_counter=0
    tmpfile=file
    file = file.replace('.png',' copy.png')
    marked_image=Image.open(file, 'r')
#     marked_pixel_values = list(marked_image.getdata())
#     width, height = marked_image.size
#     print (data[0][0])
#     if image.mode == 'RGBA':
#         channels = 4
#     elif image.mode == 'L':
#         channels = 1
#     else:
#         print("Unknown mode: %s" % image.mode)
#         return None
    marked_pixel_values = cv2.imread(file)
#     print (np.shape(marked_pixel_values))
#     print (np.shape(data))
#     print (np.shape(pixels))
#     marked_pixel_values = np.array(marked_pixel_values).reshape((height, width))
    for i in range(int(len(marked_pixel_values)/2+1)):
        for j in range(len(marked_pixel_values[0])):
            if marked_pixel_values[i][j][0] == 255 and marked_pixel_values[i][j][1] == 0 and marked_pixel_values[i][j][2] == 0 :
                true_sky_counter+=1
                if data[i][j][0] == 0 and data[i][j][1] == 0 and data[i][j][2] == 255:
                    true_positive_counter+=1
            else:
                if data[i][j][0] == 0 and data[i][j][1] == 0 and data[i][j][2] == 255:
                    false_positive_counter+=1
#                 continue
#     print ("true_positive_rate/recall for file ",tmpfile," is: ",true_positive_counter/true_sky_counter)
    true_po_list.append(true_positive_counter/true_sky_counter)
#     print ("precision for file ",tmpfile," is: ",true_positive_counter/(true_positive_counter+false_positive_counter))
    precision_list.append(true_positive_counter/(true_positive_counter+false_positive_counter))
#     print ("true sky count:", true_sky_counter)
#     print ("true positive count:",true_positive_counter)
#     print ("false positive count:",false_positive_counter)

In [57]:
def edge_cal(file):
    """
    edge detection: generate a matrix with edge pixel has value 255
    """
    from matplotlib import pyplot as plt
    file=(file+"_cropped.png")
    img = cv2.imread(file,0)
    edges = cv2.Canny(img,150,500)
    result = Image.fromarray(edges.astype(np.uint8))
    result.save(file+"edged.png")
    # print (np.shape(edges))
    # plt.subplot(121),plt.imshow(img,cmap = 'gray')
    # plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    # plt.subplot(122),plt.imshow(edges,cmap = 'gray')
    # plt.title('Edge Image'), plt.xticks([]), plt.yticks([]) 
    # plt.show()

    """
    proportion of edge pixel 
    threshold 0.1 dedicated for case which has many trees
    """

    # for i in range(len(edges)):
    #     for j in range(len(edges[i])):
    counter = 0
    for i in edges:
        for j in i:
            if j == 255:
                counter+=1
    sum=len(edges)*len(edges[0])
#     print ("edged pixels proportions for ",file, ": ",counter/sum)
    edge_list.append(counter/sum)

In [ ]:
"""
Load and feature extraction and calculation
"""

path1 = "/Users/caijieyang/Desktop/experiment"
files= listdir(path1)

for K in range(4, 16):
    true_po_list=[]
    precision_list=[]
    edge_list=[]
    # print (files)
    for file in files:
        if "panorama" in file and "cropped" not in file and "clustered" not in file and "mark" not in file and "copy" not in file:
    #         file=path1+file
            pixels,image,origin=get_image(file)
    #         print (np.shape(pixels))
                # print (pixel1)
            brightness=get_bri(pixels)
            HSV=hsv(image)
            HLS=hls(image)
            label2=K_means(file,K)
#             edge_cal(file)
            brightness=np.asarray(brightness)
            mark_sky(pixels,label2,origin,file)
    print ("K: ",K)
    print ("average true positive rate: ",np.mean(true_po_list))
    print ("average precision rate: ",np.mean(precision_list))
#     print ("average edge rate: ",np.mean(edge_list))
    print ("####################################################")
    #         print (np.shape(HSV))
    #         print (np.shape(HLS))
    

K:  4
average true positive rate:  0.900946157633072
average precision rate:  0.7664737679869986
####################################################
K:  5
average true positive rate:  0.969562987281984
average precision rate:  0.7683485673440559
####################################################


In [ ]:
print (true_po_list)

In [ ]:
# print (pixels[100,200])
# print (HLS[0,:])

In [ ]:
# """
# convert brightness into array
# """

# print (np.shape(pixels))
# print (np.shape(image))


# brightness=brightness.transpose()
# print (np.shape(brightness))
# print ("pixel RGBA: ", pixels[0,0])
# print ("cloud brightness:", brightness[0][0])
# print ("cloud RGBA:", pixels[0,0])
# print ("sky brightness:", brightness[94][462])
# print ("sky RGBA:", pixels[94,462])
# print ("tree brightness:", brightness[140][127])
# print ("tree RGBA:", pixels[140,127])

In [ ]:
# """
# convert img into grayscale
# """
# # grayimg = np.array(img)
# grayimg=cv2.imread(file)
# graypix = cv2.cvtColor(grayimg, cv2.COLOR_RGB2GRAY)

# # graypix=cvtColor(img,COLOR_RGB2GRAY)
# cv2.imwrite('gray_image.png',graypix)


In [ ]:
# print (np.shape(res2))
# print (np.shape(label2))
# print (label2[:,0])

In [ ]:

# print (HLS[115][461])
# print (pixels[36][176])

In [ ]:
#HLS [0]:H [1]:L [2]:S

# print (np.shape(Skypre))            
# print (counter)
# print (counterclass_skypre)


        
            

In [ ]:
# """
# mark based soly on blue
# """
# RED = 0
# GREEN = 1
# BLUE = 2

# data = np.asarray(image,dtype="int32")
# print (np.shape(data))
# for i in range(len(brightness)):
#     for j in range(len(brightness[0])):
#         if 100<pixels[i][j][1]<255 and pixels[i][j][0]>100 and pixels[i][j][2]>100:
#             data[i, j, GREEN] = 0
#             data[i, j, RED] = 0
#             data[i, j, BLUE] = 255
#         else:
#             continue
# result = Image.fromarray(data.astype(np.uint8))
# result.save("sky_mark1.png")

In [ ]:
# """
# transfer image into grayscale
# """

# import numpy as np
# import matplotlib.pyplot as plt

# from skimage import measure


# # Construct some test data
# # x, y = np.ogrid[-np.pi:np.pi:100j, -np.pi:np.pi:100j]
# # r = np.sin(np.exp((np.sin(x)**3 + np.cos(y)**2)))

# # Find contours at a constant value of 0.8
# contours = measure.find_contours(img, 0.8)

# # Display the image and plot all contours found
# fig, ax = plt.subplots()
# ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)

# for n, contour in enumerate(contours):
#     ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

# ax.axis('image')
# ax.set_xticks([])
# ax.set_yticks([])
# plt.show()